In [9]:
import pandas as pd
# Importing airquality module
from airquality import AirQuality

In [ ]:
# Defining city id's
city1_ids = [855, 19840, 1195]

In [ ]:
air_measurements = []
# Iterating through sensor locations and creating object to store data for each
for location in city1_ids:
    air_measurements.append(AirQuality(location))